In [1]:
#IMPORT DEPENDENCIES
import sys, requests
import pandas as pd
from pprint import pprint
from sqlalchemy import create_engine

In [ ]:
# Read the Netflix data as a pandas dataframe
data_table = "./Resources/netflix_titles.csv"
netflix_df = pd.read_csv(data_table)


# The user will have to use their own API key when running this code; I pulled our API keys
# in from another file source, shown below
sys.path.insert(1, '/Users/symbioticpenguin/Documents')
from api_keys import comdb, tomdb, tiania_api, tiania_OMDB, jomdb


# Filtering for movies only
filtered_flix = netflix_df[netflix_df["type"]=="Movie"]

# Filtering for only movies made in the United States
filtered_flix = filtered_flix[netflix_df["country"] == "United States"]

# Dropping all NaN rows
filtered_flix = filtered_flix.dropna()

# limiting to 1000 entries because free api calls only allow 1000 :(
filtered_flix = filtered_flix[:1000]

# LeTs TaKe A lOoK aT tHe DaTa
filtered_flix.head()

In [2]:
# This is the test code we used before calling the API 1,000 times!

# movie = "t=" + "Finding Nemo"
# key = "&apikey="+comdb
# url = 'http://www.omdbapi.com/?' + movie +  key
# response = requests.get(url).json()

In [3]:
# This cell contains the for loop to get all the data for movies in netflix in the omdb api

# Creating the list of movie titles to make the API calls on
filtered_movies = filtered_flix['title']

# initializing empty lists to append to in the loop
metascore = []
imdbrating = []
imdbvotes = []
boxoffice = []
title = []

# This is the api call for our OMDB dataframe
for item in filtered_movies:
    
    # If the loop doesn't work, we will skip the API call and list append function.
    try:
        # Creating the string for the URL to call the API from.
        movie = "t=" + item
        key = "&apikey="+jomdb
        url = 'http://www.omdbapi.com/?' + movie +  key
        
        # Storing the response as a json object
        response = requests.get(url).json()
        
        # Append the items we care about from the json
        boxoffice.append(response['BoxOffice'])
        title.append(response['Title'])
        metascore.append(response["Metascore"])
        imdbrating.append(response['imdbRating'])
        imdbvotes.append(response['imdbVotes'])

    except:
        next

In [4]:
# Creating the DataFrame for the OMDB data
omdb_df = pd.DataFrame({'title':title,
                        'metascore':metascore,
                        'imdb_rating':imdbrating,
                        'imdb_votes':imdbvotes,
                        'box_office':boxoffice})

In [ ]:
# Joining the two tables: the netflix table and the OMDB table
final_table = filtered_flix.join(omdb_df.set_index('title'),on = 'title', how = 'left')

In [ ]:
# Doing away with the rows that contain "NaN" in any column.
final_table = final_table.dropna()

# Write final table to csv for use later
final_table.to_csv('final_table.csv',index = False, header = True, sep = ',')

In [2]:
final_table=pd.read_csv('final_table.csv')

final_table.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,metascore,imdb_rating,imdb_votes,box_office
0,80060297,Movie,Manhattan Romance,Tom O'Brien,"Tom O'Brien, Katherine Waterston, Caitlin Fitz...",United States,"September 8, 2017",2014,TV-14,98 min,"Comedies, Independent Movies, Romantic Movies",A filmmaker working on a documentary about lov...,NaN,5.5,"1,122",NaN
1,70304988,Movie,Stonehearst Asylum,Brad Anderson,"Kate Beckinsale, Jim Sturgess, David Thewlis, ...",United States,"September 8, 2017",2014,PG-13,113 min,"Horror Movies, Thrillers","In 1899, a young doctor arrives at an asylum f...",52.0,6.8,"48,116",NaN
2,80057700,Movie,The Runner,Austin Stark,"Nicolas Cage, Sarah Paulson, Connie Nielsen, W...",United States,"September 8, 2017",2015,R,90 min,"Dramas, Independent Movies",A New Orleans politician finds his idealistic ...,39.0,4.7,"4,715",NaN
3,80045922,Movie,6 Years,Hannah Fidell,"Taissa Farmiga, Ben Rosenfield, Lindsay Burdge...",United States,"September 8, 2015",2015,NR,80 min,"Dramas, Independent Movies, Romantic Movies",As a volatile young couple who have been toget...,53.0,5.6,"10,578",NaN
4,80058026,Movie,Hell and Back,"Tom Gianas, Ross R. Shuman","Nick Swardson, Mila Kunis, Bob Odenkirk, T.J. ...",United States,"September 6, 2018",2015,R,86 min,"Action & Adventure, Comedies, Independent Movies","When best friends break a blood oath, one of t...",NaN,5.4,"4,560","$156,166"


In [3]:
#connect to local data base
rds_connection_string = "etl:etl@localhost:5432/ETLmovies_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [7]:
#check for tables 
engine.table_names()

['final_table_db']

In [6]:
#upload data frame to database
final_table.to_sql(name='final_table_db', con=engine, if_exists='append', index=False)

In [8]:
#check the dataframe made it to the table
pd.read_sql_query('select * from final_table_db', con=engine).head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,metascore,imdb_rating,imdb_votes,box_office
0,80060297,Movie,Manhattan Romance,Tom O'Brien,"Tom O'Brien, Katherine Waterston, Caitlin Fitz...",United States,"September 8, 2017",2014,TV-14,98 min,"Comedies, Independent Movies, Romantic Movies",A filmmaker working on a documentary about lov...,NaN,5.5,"1,122",None
1,70304988,Movie,Stonehearst Asylum,Brad Anderson,"Kate Beckinsale, Jim Sturgess, David Thewlis, ...",United States,"September 8, 2017",2014,PG-13,113 min,"Horror Movies, Thrillers","In 1899, a young doctor arrives at an asylum f...",52.0,6.8,"48,116",None
2,80057700,Movie,The Runner,Austin Stark,"Nicolas Cage, Sarah Paulson, Connie Nielsen, W...",United States,"September 8, 2017",2015,R,90 min,"Dramas, Independent Movies",A New Orleans politician finds his idealistic ...,39.0,4.7,"4,715",None
3,80045922,Movie,6 Years,Hannah Fidell,"Taissa Farmiga, Ben Rosenfield, Lindsay Burdge...",United States,"September 8, 2015",2015,NR,80 min,"Dramas, Independent Movies, Romantic Movies",As a volatile young couple who have been toget...,53.0,5.6,"10,578",None
4,80058026,Movie,Hell and Back,"Tom Gianas, Ross R. Shuman","Nick Swardson, Mila Kunis, Bob Odenkirk, T.J. ...",United States,"September 6, 2018",2015,R,86 min,"Action & Adventure, Comedies, Independent Movies","When best friends break a blood oath, one of t...",NaN,5.4,"4,560","$156,166"
